In [1]:
import findspark
findspark.init()

#1a01e0e77d803e8e292dbb639aa77646d41947892fe6f227

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://a92682f5ddcc:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/04 03:28:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from grandeInformacion.traffic_query_listener import TrafficListener

listener = TrafficListener()
spark.streams.addListener(listener)

In [4]:
from grandeInformacion.spark_utils import SparkUtils

logs = spark \
                .readStream \
                .format("text") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/archive") \
                .load("/home/jovyan/notebooks/data/logs_output/")

In [5]:
from pyspark.sql.functions import split, trim


logs = logs.withColumn("value", trim(logs["value"]))

logs_df = logs.select(split(logs.value, "\\|").alias("log_array"))


logs_df = logs_df.withColumn("date", logs_df["log_array"].getItem(0).cast("string"))
logs_df = logs_df.withColumn("type", logs_df["log_array"].getItem(1).cast("string"))
logs_df = logs_df.withColumn("message", logs_df["log_array"].getItem(2).cast("string"))
logs_df = logs_df.withColumn("node", logs_df["log_array"].getItem(3).cast("string"))

logs_df.printSchema()


root
 |-- log_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- date: string (nullable = true)
 |-- type: string (nullable = true)
 |-- message: string (nullable = true)
 |-- node: string (nullable = true)



In [6]:
logs_df = logs_df.filter(logs_df["type"] == "ERROR")

In [7]:
query = logs_df \
    .writeStream \
    .outputMode("append") \
    .trigger(processingTime='25 seconds') \
    .option("truncate", "false") \
    .format("console") \
    .start()

query.awaitTermination(200)

25/05/04 03:31:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7a69e226-d611-49d7-a676-96621f20377d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/05/04 03:31:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Query started: 915962bc-b973-4b57-9f0b-5aca12c5ad67


-------------------------------------------
Batch: 0
-------------------------------------------
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|log_array                                                             |date               |type |message                  |node         |
+----------------------------------------------------------------------+-------------------+-----+-------------------------+-------------+
|[04/05/2025 03:30:43, ERROR, 500 Internal Server Error, server-node-2]|04/05/2025 03:30:43|ERROR|500 Internal Server Error|server-node-2|
|[04/05/2025 03:30:47, ERROR, 500 Internal Server Error, server-node-2]|04/05/2025 03:30:47|ERROR|500 Internal Server Error|server-node-2|
|[04/05/2025 03:30:51, ERROR, 500 Internal Server Error, server-node-2]|04/05/2025 03:30:51|ERROR|500 Internal Server Error|server-node-2|
|[04/05/2025 03:31:13, ERROR, 500 Internal Server Error, server-node-

False

In [8]:
query.stop()

Query terminated: 915962bc-b973-4b57-9f0b-5aca12c5ad67


In [9]:
sc.stop()